# Estimating the Local Lipschitz Constant of a Neural Network Using Jacobinet

## Introduction
In this tutorial, we will estimate the local Lipschitz constant of a neural network using the Jacobian matrix and explore how this constant relates to the network's robustness. A neural network's Lipschitz constant bounds the rate at which its outputs can change with respect to small input perturbations. Understanding and controlling this constant is critical for:

- Adversarial robustness: Ensuring the network resists small, intentional perturbations.
- Stability: Preventing large output changes due to minor input variations.
- Generalization: Improving the network's performance on unseen data.

We will use the *Jacobinet* library (based on Keras) to calculate the Jacobian and maximize the $L_p$ norm of the gradient 
to estimate the Lipschitz constant. This provides a lower bound for the Lipschitz constant, a key metric in robustness evaluation.


# 1. Building the Neural Network
We start by defining a simple feedforward neural network with two dense layers and a ReLU activation function.

In [2]:
import keras
from keras.models import Model
from keras.layers import Dense, Activation, Input

def build_model():
    input_ = Input((10,))
    x = Dense(10, name='Dense1')(input_)
    x = Activation('relu', name='ReLU1')(x)
    x = Dense(10)(x)
    x = Activation('relu')(x)
    output = Dense(2, name='Output')(x)
    return Model(input_, output)

model = build_model()
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 10)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dense1 (Dense)                  │ (None, 10)             │           110 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ ReLU1 (Activation)              │ (None, 10)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           110 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 10)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Output (Dense)                  │ (None, 2)              │            22 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 242 (968.00 B)

 Trainable params: 242 (968.00 B)

 Non-trainable params: 0 (0.00 B)

## 2. Computing the Jacobian with Jacobinet

We will now compute the Jacobian matrix using Jacobinet’s get_backward_model function. 
This model returns the gradient of each output with respect to the input.

In [4]:
import jacobinet
from jacobinet import clone_to_backward
import numpy as np

# Placeholder gradient to compute the Jacobian
gradient_placeholder = keras.Variable(np.ones((1, 2)))

# Compute backward model for Jacobian calculation
backward_model = clone_to_backward(model, gradient=gradient_placeholder)


### Explanation:

The Jacobian represents the gradients of the output w.r.t. the input.
get_backward_model builds a model to compute these gradients.

# 3. Estimating the Lipschitz Constant
To compute the Lipschitz constant, we use the L2 norm (p=2) of the Jacobian.

In [5]:
from jacobinet import get_lipschitz_model

# Create a Lipschitz model using the L2 norm (p=2)
lipschitz_model = get_lipschitz_model(backward_model, p=2)
lipschitz_model.summary()


Model: "lipschitz_model"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 10)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Dense1 (Dense)      │ (None, 10)        │        110 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ grad_constant_1     │ (1, 2)            │          0 │ input_layer_1[0]… │
│ (GradConstant)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ ReLU1 (Activation)  │ (None, 10)        │          0 │ Dense1[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ backward_dense_3    │ (1, 10)           │         22 │ grad_constant_1[… │
│ (BackwardDense)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 10)        │        110 │ ReLU1[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ backward_activatio… │ (1, 10)           │          0 │ backward_dense_3… │
│ (BackwardActivatio… │                   │            │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ backward_dense_4    │ (1, 10)           │        110 │ backward_activat… │
│ (BackwardDense)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ backward_activatio… │ (1, 10)           │          0 │ backward_dense_4… │
│ (BackwardActivatio… │                   │            │ Dense1[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ backward_dense_5    │ (1, 10)           │        110 │ backward_activat… │
│ (BackwardDense)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lipschitz           │ (1)               │          0 │ backward_dense_5… │
│ (Lipschitz)         │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 242 (968.00 B)

 Trainable params: 242 (968.00 B)

 Non-trainable params: 0 (0.00 B)

## Visualizing the Model Structure


In [6]:

import keras.utils
from IPython.display import HTML

dot_img_file_lipschitz = './model_dense_lipschitz.png'
keras.utils.plot_model(lipschitz_model, to_file=dot_img_file_lipschitz, show_shapes=True, show_layer_names=True)
HTML('<div style="text-align: center;"><img src="{}" width="400"/></div>'.format(dot_img_file_lipschitz))


# 4. Evaluating the Lipschitz Constant on Random Data
We now evaluate the Lipschitz constant using random input data.

In [7]:
data = np.asarray(np.random.rand(10)[None], dtype='float32') # Generate random input data

# Compute the lower bound of the Lipschitz constant
lipschitz_constant = lipschitz_model(data)
print(f"The Lipschitz constant is at least: {lipschitz_constant}")

The Lipschitz constant is at least: tensor([0.3400], device='mps:0', grad_fn=<SqrtBackward0>)


In [8]:
lipschitz_threshold = lipschitz_constant[0]

# 5. Maximizing the Lp Norm with Adversarial Attacks (PGD)
We use Projected Gradient Descent (PGD) to iteratively perturb the input and maximize the Lp norm, tightening the lower bound of the Lipschitz constant.

In [9]:
import torch
import torchattacks
import torch.nn as nn

class LipAttack(nn.Module):
    def __init__(self, keras_model):
        super().__init__()
        self.keras_model = keras_model
        self.lipschitz_threshold = keras.Variable(lipschitz_threshold.cpu().detach().numpy())

    def forward(self, x):
        x = self.keras_model(x)
        return torch.cat([keras.ops.relu(self.lipschitz_threshold-x), keras.ops.relu(x-self.lipschitz_threshold)], -1)

# Wrap lipschitz_model with the attack class
torch_lip_model = LipAttack(lipschitz_model)

# Apply PGD attack for different iteration steps
adv_data = data
for steps in [10, 20, 40, 100, 200, 1000]:
    lip_attack = torchattacks.PGD(torch_lip_model, eps=10., steps=steps)
    adv_data_ = lip_attack(torch.Tensor(adv_data), torch.Tensor([1, 0]))
    lipschitz_constant_adv = lipschitz_model(adv_data)

    if torch_lip_model(adv_data_).argmax().cpu().detach().numpy()==1:
        adv_data = adv_data_
        torch_lip_model.lipschitz_threshold.assign(lipschitz_constant_adv[0].cpu().detach().numpy())
    print(f"Lipschitz constant after {steps} PGD steps: {lipschitz_constant_adv}")


Lipschitz constant after 10 PGD steps: tensor([0.3400], device='mps:0', grad_fn=<SqrtBackward0>)
Lipschitz constant after 20 PGD steps: tensor([0.3400], device='mps:0', grad_fn=<SqrtBackward0>)
Lipschitz constant after 40 PGD steps: tensor([0.4505], device='mps:0', grad_fn=<SqrtBackward0>)
Lipschitz constant after 100 PGD steps: tensor([0.5222], device='mps:0', grad_fn=<SqrtBackward0>)
Lipschitz constant after 200 PGD steps: tensor([0.7852], device='mps:0', grad_fn=<SqrtBackward0>)
Lipschitz constant after 1000 PGD steps: tensor([0.7852], device='mps:0', grad_fn=<SqrtBackward0>)


## 6. Theoretical Insights: Why Lipschitz Constant Matters

### 1. Adversarial Robustness
A smaller Lipschitz constant implies that the network’s output changes less when small perturbations are applied to the input. This makes it more resistant to **adversarial attacks**, where maliciously crafted input perturbations attempt to mislead the model.

### 2. Stability and Generalization
Networks with lower Lipschitz constants tend to generalize better, as they are less sensitive to noise or variations in input data. This also enhances training stability, as it prevents excessive variations in gradients.

### 3. Mathematical Context
The Lipschitz constant \( L \) is formally defined as:


$$L = \sup_{x \neq y} \frac{\|f(x) - f(y)\|}{\|x - y\|} $$


Locally, this can be approximated using the **Jacobian matrix** \( J(x) \), which contains all first-order partial derivatives of the network’s outputs with respect to its inputs:

$$ L = \max_{x} \|J(x)\|_p $$


where \( \|J(x)\|_p \) is the **Lp norm** of the Jacobian matrix. Maximizing this norm provides a lower bound on the Lipschitz constant, a key metric for evaluating the robustness of the neural network.


# Conclusion

In this tutorial, we used Jacobinet to compute a lower bound for the Lipschitz constant by maximizing the Lp norm using adversarial attacks. The Lipschitz constant is a key measure of a network's robustness and generalization capabilities.